In [1]:
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import requests
import pandas as pd
import seaborn as sns

In [2]:

url = 'https://www.automobile.tn/fr/occasion'
nbre_page=2
n=1 #page 1
total_df = pd.DataFrame()

In [3]:
roads=[]
years=[]
boites=[]
transmissions=[]
horsepowers=[]
fuels=[]
nbre_portes=[]
cylindres=[]
items = []
marques = []
modeles = []
prices = []
links = []

In [4]:
for n in range(nbre_page):

    url = 'https://www.automobile.tn/fr/occasion/'+str(n)
    code_page=requests.get(url)

    print(url)
    html=code_page.text

    soup= BeautifulSoup(html, 'html.parser')

    container=soup.findAll(class_='occasion-item-v2')
    for item in container:
        link_tag = item.find('a', class_='occasion-link-overlay', href=True)
        if link_tag:
            link = link_tag['href']
            correct_link='https://www.automobile.tn/'+link
            links.append(correct_link)
    for link in links:
        code_page=requests.get(link)
        html=code_page.text
        soup= BeautifulSoup(html, 'html.parser')
        container_a=soup.findAll(class_='spec-value text-end') 
        for item in container_a:
            nbre_portes.append(item.text.strip())
        
    print(nbre_portes)

    for item in container:
        road=item.find('li',class_='road').text.strip()
        roads.append(road)
        print(road)

        year=item.find('li',class_='year').text.strip()
        years.append(year)
        print(year)

        boite=item.find('li',class_='boite').text.strip()
        boites.append(boite)
        print(boite)

        transmission=item.find('li',class_='transmission').text.strip()
        transmissions.append(transmission)
        print(transmission)

        horsepower=item.find('li',class_='horsepower').text.strip()
        horsepowers.append(horsepower)
        print(horsepower)

        fuel=item.find('li',class_='fuel').text.strip()
        fuels.append(fuel)
        print(fuel)



        #extraire la marque et le modele(sous h2)
        noms = item.findAll('h2')
        for nom in noms:
            nom_car = nom.text.strip()
            if nom_car:
                items.append(nom_car)
                marques.append(nom_car.split()[0])
                parts = nom_car.split()
                modeles.append(' '.join(parts[1:]))
        prices_dirty=item.findAll('div',class_='price')
        for p in prices_dirty:
            if p:
                price_final = p.text.replace('DT', '').replace('\xa0', '').strip()
                prices.append(price_final)
        
    print(prices)
    if len(prices) == len(marques) == len(modeles) == len(roads) == len(years) == len(boites) == len(transmissions) == len(horsepowers) == len(fuels):
        data = {'Marque': marques, 'Modèle': modeles, 'road': roads, 'year': years, 'boite': boites, 'transmission': transmissions, 'horsepower': horsepowers, 'fuel': fuels, 'Prix': prices}
        df = pd.DataFrame(data)
        print(df)
    
    total_df = pd.concat([total_df, df], ignore_index=True)

    print('-------------------------------------------------------------------------------------------------')
    print(nbre_portes)


https://www.automobile.tn/fr/occasion/0
['MG', 'RX8', 'Noir', 'Marron', 'Cuir partiel', '5', '5', 'Essence', 'Automatique', '16                                                    cv', 'Traction', '2000                                                    cm³', 'BMW', 'Série 3', '(F30) Berline01/2011 - 08/2015', 'Noir', 'Noir', 'Tissu', '5', '5', '316i 1.6 16V', '136                                                    ch dyn', 'Essence', 'Manuelle', '7                                                    cv', 'Traction', '3160                                                    cm³', 'Land Rover', 'Range Rover Sport', 'Sport I06/2005 - 08/2009', 'Blanc', 'Beige', 'Similicuir', '5', '4', 'Diesel', 'Automatique', '13                                                    cv', 'Traction', '2689                                                    cm³', 'Mahindra', 'KUV 100', "01/2016 - Aujourd'hui", 'Blanc', 'Noir', 'Tissu', '5', '5', '1.2 82cv', '82                                                    

In [5]:

#la dataframe totale contenant les données scrapées de toutes les pages:
total_df.to_csv('../data/cars.csv', index=False)
# print(df)
print(links)

['https://www.automobile.tn//fr/occasion/kia/rio/108929', 'https://www.automobile.tn//fr/occasion/bmw/serie-4-gran-coupe/108313', 'https://www.automobile.tn//fr/occasion/kia/picanto/109162', 'https://www.automobile.tn//fr/occasion/chery/tiggo-3x/106792', 'https://www.automobile.tn//fr/occasion/hyundai/grand-i10/109161', 'https://www.automobile.tn//fr/occasion/mercedes-benz/classe-c/108227', 'https://www.automobile.tn//fr/occasion/peugeot/partner-origin-utilitaire/108228', 'https://www.automobile.tn//fr/occasion/volkswagen/golf-8/108223', 'https://www.automobile.tn//fr/occasion/wallyscar/iris/108458', 'https://www.automobile.tn//fr/occasion/wallyscar/719/106855', 'https://www.automobile.tn//fr/occasion/audi/a3-sportback/107031', 'https://www.automobile.tn//fr/occasion/hyundai/tucson/108969', 'https://www.automobile.tn//fr/occasion/kia/rio/108929', 'https://www.automobile.tn//fr/occasion/bmw/serie-4-gran-coupe/108313', 'https://www.automobile.tn//fr/occasion/kia/picanto/109162', 'https:/